<center><div class="alert alert-block alert-info" style=" line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px; color:green"> &nbsp; Bengali.AI Speech Recognition : Our very first submission</b><br><br><b style="font-size: 18px; color:green">HELLO EVERYONE!</b><br>
</div></center>

In this Notebook we'll try to submit our first submission by using a publicly available model.

We'll be using this [BanglaASR](https://huggingface.co/bangla-speech-processing/BanglaASR) model. It is actually a finetuned [whisper](https://openai.com/research/whisper) model on rained [Bangla Mozilla Common Voice Dataset](https://arxiv.org/abs/2206.14053). It's reported performance is 4.58% on 7k validation set which seems pretty good to be a starting point. So let's start with that.

I'll be trying to use the other publicly available models one by one and see how they perform. Then we'll get an idea from where we can start the training/finetuning on this huge dataset.

# Imports

In [3]:
pip install transformers

In [4]:
pip install pydub

In [5]:
import os
import librosa
import torch
import torchaudio
import numpy as np

from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperForConditionalGeneration
from tqdm import tqdm
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

# Load the model

While submitting notebooks, you need to have internet disabled. So you can't download models from huggingface directly. One workaround is to download the models from huggingface, upload them to kaggle as datasets and then use them!

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "bangla-speech-processing/BanglaASR"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

# Demo inference

We'll write an inference function where we'll give the path as input and it will return the transcription!

First we'll need to convert the audio sampling rate to 16k since WhisperFeatureExtractor requires audios to be sampled at 16k.

In [7]:
#This code is modified from the source code provided here https://huggingface.co/bangla-speech-processing/BanglaASR
def inference_fn(path):
    speech_array, sampling_rate = sf.read(mp3_path)
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription


In [8]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [9]:
mp3_path = "/content/drive/MyDrive/Habiganj_Audio/all_audios/rec_01_audio_10.wav"
print(f"File name :",mp3_path)
AudioSegment.from_file(mp3_path)

File name : /content/drive/MyDrive/Habiganj_Audio/all_audios/rec_01_audio_10.wav


In [ ]:
import os
test_path = "/content/drive/MyDrive/Habiganj_Audio/all_audios/"
files = os.listdir(test_path)
ids = []
sentences = []
for file in tqdm(files):
    ids.append(file.split(".")[0])
    mp3_path = os.path.join(test_path,file)
    prediction = inference_fn(mp3_path)

    #sanity check
    if len(prediction)==0:
        prediction = "<>"

    sentences.append(prediction)



  0%|          | 0/1184 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame({"External_ID":ids,"contents":sentences})
df.head()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.to_csv("habiganj-banglaWhisper-inferences.csv",index=False)

In [ ]:
df.to_excel("habiganj-banglaWhisper-inferences.xlsx")

In [ ]:
print('done')